# 🚆Train_station
지하철 정류장 데이터 전처리

### TODO
- [X] 데이터 불러오기, 필요없는 로우 및 컬럼 일차 삭제  
- [X] 결측값 확인
  - [X] 위경도 결측값 분석
  - [X] 결측값 채워진 데이터셋 구하기
  - [X] 노선명 통일
  - [X] 역사명 통일
- [X] 두 데이터셋 하나로 결합
  - [X] 역사명 통일
  - [X] 노선명 통일 
  - [X] 위경도 이상값 확인 
  - [X] 위경도 결측값 재확인
- [X] 지도에 시각화
  - [X] 이상값 확인
- [ ] 역번호, 노선번호 맞추기
  - [X] 서울시 지하철 실시간 도착정보에 쓰이는 역 종류 확인
  - [X] 전체 지하철 시간표 정보에 쓰이는 역 종류 확인
  - [ ] 역번호 통일
  - [ ] 노선번호 통일
- [ ] 수도권 바깥 데이터 삭제
  - [ ] 주소 데이터 결측값 확인
  - [ ] 결측값 채워진 데이터셋 구하기
  - [ ] 이상값 확인
- [ ] 환승역, 종점, 분기역 정보 가공

### 0. import

In [632]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from IPython.display import set_matplotlib_formats
import koreanize_matplotlib
import folium

### 1. 데이터 불러오기, 필요없는 로우 및 컬럼 일차 삭제  

In [633]:
# 데이터 읽어오기
station = pd.read_csv('train_station.csv')

# 확실히 필요없는 컬럼 제거
station = station.drop(columns=['영문역사명', '한자역사명', '운영기관명', '역사도로명주소', '역사전화번호','데이터기준일자'])

# 확실히 필요없는 행 제거
# # 인천 1~2호선
# station = station[(station['노선번호'].str.slice(stop=3) != 'S28')]
# 인천 자기부상철도 -> 휴업 상태
station = station[(station['노선번호'] != 'S28M1')]
# 대전
station = station[(station['노선번호'].str.slice(start=1, stop=3) != '30')]
# 대구 1~3호선
station = station[(station['노선번호'].str.slice(start=1, stop=3) != '27')]
# 부산 1~4호선, 김해선
station = station[(station['노선번호'].str.slice(start=1, stop=3) != '26') & (station['노선번호'] != 'L48B1')]
# 광주
station = station[(station['노선번호'].str.slice(start=1, stop=3) != '29')]

# # 나중을 위해 역번호 저장
# station_id = station['역번호']

station = station.reset_index(drop=True)
station

,역번호,역사명,노선번호,노선명,환승역구분,환승노선번호,환승노선명,역위도,역경도
0,3110,계양,S2801,인천지하철 1호선,환승역,S2801+I28A1,공항철도,37.571539,126.736319
1,3111,귤현,S2801,인천지하철 1호선,일반역,-,-,37.566362,126.742498
2,3112,박촌,S2801,인천지하철 1호선,일반역,-,-,37.553525,126.744946
3,3113,임학,S2801,인천지하철 1호선,일반역,-,-,37.545058,126.738642
4,3114,계산,S2801,인천지하철 1호선,일반역,-,-,37.543243,126.728436
...,...,...,...,...,...,...,...,...,...
748,4811,달미역,I41WS,서해선,일반역,NaN,NaN,37.349320,126.808218
749,4812,선부역,I41WS,서해선,일반역,NaN,NaN,37.333908,126.810977
750,4813,초지역,I41WS,서해선,환승역,"I4103, I28K1","수인선, 과천안산선",37.320583,126.806151
751,4814,시우역,I41WS,서해선,일반역,NaN,NaN,37.313767,126.798303


### 2. 결측값 확인

##### 2-a. 위경도 결측값 분석

In [634]:
# 역이 노선별로 총 몇 개인지 확인
# 1호선이어도, 구간별로 노선 명칭 다름 (우리가 아는 1호선 = 경원선 + 1호선 + 경인선 + 경부선 + 장항선)
total_station_cnt_by_route = station["노선명"].value_counts().sort_index()
total_station_cnt_by_route

1호선               10
2호선               50
3호선               34
4호선               26
5호선               56
6호선               39
7호선               42
8호선               18
경강선               11
경부선               45
경원선               24
경의중앙선             57
경인선               20
경춘선               24
김포골드라인            10
도시철도 7호선          11
분당선               37
서해선               12
수도권  도시철도 9호선     25
수도권 경량도시철도 신림선    11
수인선               26
신분당선              16
안산과천선             21
에버라인              15
우이신설선             13
의정부               15
인천국제공항선           14
인천지하철 1호선         30
인천지하철 2호선         27
일산선               11
진접선                3
Name: 노선명, dtype: int64

In [635]:
# 결측값이 컬럼별로 몇 개인지 확인
station.isnull().sum()

역번호         0
역사명         0
노선번호        0
노선명         0
환승역구분       0
환승노선번호    476
환승노선명     476
역위도       352
역경도       352
dtype: int64

In [636]:
# 역위도, 역경도가 결측값인 역이 노선별로 몇 개인지 확인
null_station_cnt_by_route = station['노선명'][station['역위도'].isnull(
)].value_counts().sort_index()
null_station_cnt_by_route

1호선              10
2호선              50
3호선              34
4호선              26
5호선              56
6호선              39
7호선              42
8호선              18
김포골드라인           10
도시철도 7호선         11
수도권  도시철도 9호선    25
우이신설선            13
의정부              15
진접선               3
Name: 노선명, dtype: int64

##### 2-b. 결측값 채워진 데이터셋 구하기

In [637]:
# 1. 노선별 전체 역 개수가 실제 역 개수와 다름
# 2. 위경도 결측 데이터가 존재하는 노선에 대해 알아본 결과, 서울시 역사마스터 정보와 제대로 합처지지 않은 것으로 보임
# -> 서울시 역사마스터 정보와의 결합 필요

# 서울시 역사마스터 데이터 읽어오기
station_seoul = pd.read_csv('train_station_seoul.csv')

# 기존 데이터셋과 컬럼 이름 맞추기
station_seoul.rename(columns={'역사_ID': '역번호', '호선': '노선명'}, inplace=True)
station.rename(columns={'역위도': '위도', '역경도': '경도'}, inplace=True)

# 서울시 역사마스터 데이터, 위도와 경도가 반대로 들어가 있는 문제 해결
station_seoul.rename(columns={'위도': '경도', '경도': '위도'}, inplace=True)
station_seoul = station_seoul[['역번호', '역사명', '노선명', '위도', '경도']]

# 필요없는 노선 제거
station_seoul = station_seoul[(station_seoul['노선명'] != '장항선')]

station_seoul

,역번호,역사명,노선명,위도,경도
0,9996,미사,5호선,37.560927,127.193877
1,9995,강일,5호선,37.557490,127.175930
2,4929,김포공항,김포골드라인,37.562360,126.801868
3,4928,고촌,김포골드라인,37.601243,126.770345
4,4927,풍무,김포골드라인,37.612488,126.732387
...,...,...,...,...,...
760,154,종로5가,1호선,37.570926,127.001849
761,153,종로3가,1호선,37.570406,126.991847
762,152,종각,1호선,37.570161,126.982923
763,151,시청,1호선,37.565715,126.977088


In [638]:
# 역이 노선별로 몇 개인지 확인
total_station_seoul_cnt_by_route = station_seoul["노선명"].value_counts().sort_index()
total_station_seoul_cnt_by_route

# 확인 결과, 앞서 전체 역정보 데이터셋에 빠진 일부 역이 이 데이터셋에는 포함되어 있음 (역 개수 더 많음)

1호선          10
2호선          50
3호선          34
4호선          26
5호선          58
6호선          39
7호선          53
7호선(인천)       9
8호선          18
9호선          25
9호선(연장)      13
경강선          11
경부선          39
경원선          30
경의중앙선        31
경인선          20
경춘선          19
공항철도1호선      14
과천선           9
김포골드라인       10
분당선          35
서해선          12
수인선          18
신림선          11
신분당선          6
신분당선(연장)      7
신분당선(연장2)     3
안산선          13
에버라인선        15
우이신설선        13
의정부선         15
인천1호선        30
인천2호선        27
일산선          11
중앙선          21
진접선           3
Name: 노선명, dtype: int64

In [639]:
# 역위도, 역경도가 결측값인 역이 노선별로 몇 개인지 확인
station_seoul.isnull().sum()

# 확인 결과, 해당 데이터셋은 위경도 정보가 빠진 것 없이 잘 들어있음 

역번호    0
역사명    0
노선명    0
위도     0
경도     0
dtype: int64

##### 2-c. 노선명 통일

In [640]:
# 둘 중 한 데이터셋에만 있는 노선명 확인
route_name = set(total_station_cnt_by_route.index)
route_name_seoul = set(total_station_seoul_cnt_by_route.index)
sub_route_name = route_name - route_name_seoul
sub_route_name_seoul = route_name_seoul - route_name

print(sub_route_name)
print(sub_route_name_seoul)

{'수도권  도시철도 9호선', '안산과천선', '인천지하철 1호선', '인천지하철 2호선', '에버라인', '인천국제공항선', '수도권 경량도시철도 신림선', '도시철도 7호선', '의정부'}
{'신림선', '인천2호선', '7호선(인천)', '공항철도1호선', '신분당선(연장2)', '안산선', '과천선', '의정부선', '9호선(연장)', '신분당선(연장)', '인천1호선', '중앙선', '에버라인선', '9호선'}


In [641]:
# 노선명 통일
station["노선명"] = station["노선명"].replace({
    "도시철도 7호선": "7호선",
    "수도권  도시철도 9호선": "9호선",
    "수도권 경량도시철도 신림선": "신림선",
    "인천지하철 1호선": "인천1호선",
    "인천지하철 2호선": "인천2호선",
    "인천국제공항선": "공항철도선",
    "에버라인": "에버라인선",
    "의정부": "의정부선"
})

station_seoul["노선명"] = station_seoul["노선명"].replace({
    "공항철도1호선": "공항철도선",
    "안산선": "안산과천선",
    "과천선": "안산과천선",
    "9호선(연장)": "9호선",
    "7호선(인천)": "7호선",
    "신분당선(연장)": "신분당선",
    "신분당선(연장2)": "신분당선"
})

# 둘 중 한 데이터셋에만 있는 노선명 재확인
route_name = set(station["노선명"])
route_name_seoul = set(station_seoul["노선명"])
sub_route_name = route_name - route_name_seoul
sub_route_name_seoul = route_name_seoul - route_name

print(sub_route_name)
print(sub_route_name_seoul)

# 중앙선은 이후 역 종류 보고 어디에 넣을지 결정하기 

set()
{'중앙선'}


In [642]:
# 각 노선별 역 개수 확인해 보기
total_station_cnt_by_route = station["노선명"].value_counts().sort_index()
total_station_cnt_by_route

1호선       10
2호선       50
3호선       34
4호선       26
5호선       56
6호선       39
7호선       53
8호선       18
9호선       25
경강선       11
경부선       45
경원선       24
경의중앙선     57
경인선       20
경춘선       24
공항철도선     14
김포골드라인    10
분당선       37
서해선       12
수인선       26
신림선       11
신분당선      16
안산과천선     21
에버라인선     15
우이신설선     13
의정부선      15
인천1호선     30
인천2호선     27
일산선       11
진접선        3
Name: 노선명, dtype: int64

In [643]:
total_station_seoul_cnt_by_route = station_seoul["노선명"].value_counts().sort_index()
total_station_seoul_cnt_by_route

1호선       10
2호선       50
3호선       34
4호선       26
5호선       58
6호선       39
7호선       62
8호선       18
9호선       38
경강선       11
경부선       39
경원선       30
경의중앙선     31
경인선       20
경춘선       19
공항철도선     14
김포골드라인    10
분당선       35
서해선       12
수인선       18
신림선       11
신분당선      16
안산과천선     22
에버라인선     15
우이신설선     13
의정부선      15
인천1호선     30
인천2호선     27
일산선       11
중앙선       21
진접선        3
Name: 노선명, dtype: int64

##### 2-d. 역사명 통일

In [644]:
# 둘 중 한 데이터셋에만 있는 역사명 확인
station_name = set(station["역사명"])
station_name_seoul = set(station_seoul["역사명"])
sub_station_name = sorted(station_name - station_name_seoul)
sub_station_name_seoul = sorted(station_name_seoul - station_name)

print(sub_station_name)
print(sub_station_name_seoul)

['가능역', '가산디지털단지역', '가좌역', '가천대역', '가평역', '간석역', '갈매역', '강남구청역', '강매역', '강촌역', '개봉역', '개포동역', '걸포북변역', '검단오류', '경기광주역', '경마공원역', '고색역', '고잔역', '고촌역', '곡산역', '곤지암역', '공덕역', '과천역', '관악산', '관악역', '광명역', '광운대역', '교대(법원·검찰청)', '구래역', '구로역', '구룡역', '구리역', '구성역', '구일역', '국수역', '군포역', '굴봉산역', '금곡역', '금릉역', '금정역', '금천구청역', '금촌역', '기흥(백남준아트센터)', '기흥역', '김유정역', '김포공항역', '낙성대(강감찬)', '남동인더스파크역', '남영역', '남춘천역', '남한산성입구(성남법원·검찰청)', '노량진역', '녹양역', '녹천역', '능곡역', '달미역', '달월역', '당정역', '대곡역', '대공원역', '대모산입구역', '대방역', '대성리역', '대야미역', '대화역', '덕계역', '덕소역', '덕정역', '도곡역', '도농역', '도봉산역', '도봉역', '도심역', '도원역', '도화역', '독산역', '동대문역사문화공원(DDP)', '동두천역', '동두천중앙역', '동백(용인세브란스)', '동암역', '동인천역', '동작', '두정역', '디지털미디어시티역', '마두역', '마산역', '마석역', '망우역', '망월사역', '망포역', '매교역', '매탄권선역', '명학역', '모란역', '문산역', '미금역', '반월역', '방학역', '배방역', '백마역', '백석역', '백양리역', '백운역', '범계역', '별내별가람역', '별내역', '병점역', '보산역', '보정역', '복정역', '봉명역', '부개역', '부발역', '부천역', '부평역', '사릉역', '사리역', '사우(김포시청)역', '산본역', '삼동역', '삼송역', '상갈역', '상록수역', '상봉역', '상천역', '서강대

In [645]:
# 역이름 통일
# 역이름 맨 끝에 등장하는'역'자 제거
station['역사명'] = station['역사명'].str.replace(pat=r'역$', repl=r'', regex=True)
# 서울역은 다시 '역' 추가 처리 
station['역사명'] = station['역사명'].str.replace(pat=r'^서울$', repl=r'서울역', regex=True)

# 결과 다시 확인
station_name = set(station["역사명"])
station_name_seoul = set(station_seoul["역사명"])
sub_station_name = sorted(station_name - station_name_seoul)
sub_station_name_seoul = sorted(station_name_seoul - station_name)

print(sub_station_name)
print(sub_station_name_seoul)

['검단오류', '관악산', '교대(법원·검찰청)', '기흥(백남준아트센터)', '낙성대(강감찬)', '남한산성입구(성남법원·검찰청)', '동대문역사문화공원(DDP)', '동백(용인세브란스)', '동작', '배방', '봉명', '상봉', '서울역(경의선)', '성신여대입구', '수원(분당)', '시민공원', '시우', '시청·용인대', '신창(순천향대)', '신촌(지하)', '쌍용(나사렛대)', '아산', '아시아드경기장', '온수', '온양온천', '왕십리', '용마산(용마폭포공원)', '운동장·송담대(중앙시장)', '운연', '이촌', '전대·에버랜드', '종로3가(탑골공원)', '청량리', '탕정', '하남시청(덕풍∙신장)', '흑석']
['검단오류(검단산업단지)', '관악산(서울대)', '교대(법원.검찰청)', '낙성대', '남한산성입구(성남법원.검찰청)', '동대문역사문화공원', '동백', '둔촌오륜', '봉은사', '삼성중앙', '삼전', '석촌고분', '송파나루', '시민공원(문화창작지대)', '시청.용인대', '아시아드경기장(공촌사거리)', '언주', '용마산', '운동장.송담대', '운연(서창)', '원곡', '이수', '전대.에버랜드', '중앙보훈병원', '하남시청(덕풍-신장)', '한성백제', '흑석(중앙대입구)']


In [646]:
# 특수문자 .으로 통일
station['역사명'] = station['역사명'].str.replace(pat=r'[·∙-]', repl=r'.', regex=True)
station_seoul['역사명'] = station_seoul['역사명'].str.replace(
    pat=r'[·∙-]', repl=r'.', regex=True)

# 수동 맞추기 -> 괄호 생기는 방향으로 (단, 지하/지상 정보, 노선명인 경우 제거)
station["역사명"] = station["역사명"].replace({
    "동작": "동작(현충원)",
    "검단오류": "검단오류(검단산업단지)",
    "관악산": "관악산(서울대)",
    "서울역(경의선)": "서울역",
    "시민공원": "시민공원(문화창작지대)",
    "수원(분당)": "수원",
    "신촌(지하)": "신촌",
    "운연": "운연(서창)",
    "흑석": "흑석(중앙대입구)",
    "아시아드경기장": "아시아드경기장(공촌사거리)",
})

station_seoul["역사명"] = station_seoul["역사명"].replace({
    "낙성대": "낙성대(강감찬)",
    "동대문역사문화공원": "동대문역사문화공원(DDP)",
    "동백": "동백(용인세브란스)",
    "운동장.송담대": "운동장.송담대(중앙시장)",
    "용마산": "용마산(용마폭포공원)",
})

# 결과 다시 확인
station_name = set(station["역사명"])
station_name_seoul = set(station_seoul["역사명"])
sub_station_name = sorted(station_name - station_name_seoul)
sub_station_name_seoul = sorted(station_name_seoul - station_name)

print(sub_station_name)
print(sub_station_name_seoul)

['기흥(백남준아트센터)', '배방', '봉명', '상봉', '성신여대입구', '시우', '신창(순천향대)', '쌍용(나사렛대)', '아산', '온수', '온양온천', '왕십리', '이촌', '종로3가(탑골공원)', '청량리', '탕정']
['둔촌오륜', '봉은사', '삼성중앙', '삼전', '석촌고분', '송파나루', '언주', '원곡', '이수', '중앙보훈병원', '한성백제']


### 3. 두 데이터셋 하나로 결합

##### 3-a. 역사명, 노선명 하나로 합치기

In [647]:
# 일단 하나로 합쳐보기
station_result = pd.merge(station, station_seoul, on=['역사명', '노선명'], how='outer')
station_result

,역번호_x,역사명,노선번호,노선명,환승역구분,환승노선번호,환승노선명,위도_x,경도_x,역번호_y,위도_y,경도_y
0,3110,계양,S2801,인천1호선,환승역,S2801+I28A1,공항철도,37.571539,126.736319,3110.0,37.571449,126.735780
1,3111,귤현,S2801,인천1호선,일반역,-,-,37.566362,126.742498,3111.0,37.566379,126.742654
2,3112,박촌,S2801,인천1호선,일반역,-,-,37.553525,126.744946,3112.0,37.553703,126.745077
3,3113,임학,S2801,인천1호선,일반역,-,-,37.545058,126.738642,3113.0,37.545059,126.738665
4,3114,계산,S2801,인천1호선,일반역,-,-,37.543243,126.728436,3114.0,37.543238,126.728128
...,...,...,...,...,...,...,...,...,...,...,...,...
812,NaN,옥수,NaN,경원선,NaN,NaN,NaN,NaN,NaN,1011.0,37.540446,127.018672
813,NaN,한남,NaN,경원선,NaN,NaN,NaN,NaN,NaN,1010.0,37.529430,127.009169
814,NaN,서빙고,NaN,경원선,NaN,NaN,NaN,NaN,NaN,1009.0,37.519594,126.988537
815,NaN,이촌(국립중앙박물관),NaN,경원선,NaN,NaN,NaN,NaN,NaN,1008.0,37.522427,126.973406


In [648]:
# 역번호_x가 null인 값이 노선별로 몇 개 있는지 각각 확인해보기
null_x_station_cnt_by_route = station_result['노선명'][station_result['역번호_x'].isnull(
)].value_counts().sort_index()
null_x_station_cnt_by_route

5호선       1
7호선       1
9호선      13
경부선       1
경원선       7
경인선       1
서해선       1
안산과천선     1
에버라인선     1
우이신설선     1
중앙선      21
Name: 노선명, dtype: int64

In [649]:
# 역번호_y가 null인 값이 노선별로 몇 개 있는지 각각 확인해보기
null_y_station_cnt_by_route = station_result['노선명'][station_result['역번호_y'].isnull(
)].value_counts().sort_index()
null_y_station_cnt_by_route

5호선       1
7호선       1
경부선       7
경원선       1
경의중앙선    30
경인선       1
경춘선       5
분당선       2
서해선       1
수인선       8
에버라인선     1
우이신설선     1
Name: 노선명, dtype: int64

In [650]:
# 5호선
station_result[(station_result['노선명'] == '5호선') & (
    station_result['역번호_x'].isnull() | station_result['역번호_y'].isnull())]

# 역사명 합치기 
station_result.drop(785, inplace=True)
station_result.iloc[303, [9, 10, 11]] = (
    2535.0, 37.57254, 126.99030)

# 결과 확인
# 모든 역사명을 종로3가(탑골공원)로 맞추기
station_result["역사명"] = station_result["역사명"].replace({
    "종로3가": "종로3가(탑골공원)"})
station_result[station_result['역사명'] == '종로3가(탑골공원)']

,역번호_x,역사명,노선번호,노선명,환승역구분,환승노선번호,환승노선명,위도_x,경도_x,역번호_y,위도_y,경도_y
165,0130,종로3가(탑골공원),I4101,1호선,도시철도 환승역,I1103+S1105,수도권 광역철도 3호선+수도권 도시철도 5호선,NaN,NaN,153.0,37.570406,126.991847
229,0329,종로3가(탑골공원),I1103,3호선,도시철도 환승역,S1105+I1101,수도권 도시철도 5호선+수도권 광역철도 1호선,NaN,NaN,319.0,37.571605,126.991791
303,0534,종로3가(탑골공원),S1105,5호선,도시철도 환승역,I1103+I1101,수도권 광역철도 3호선+수도권 광역철도 1호선,NaN,NaN,2535.0,37.572540,126.990300


In [651]:
# 7호선
station_result[(station_result['노선명'] == '7호선') & (
    station_result['역번호_x'].isnull() | station_result['역번호_y'].isnull())]

# 역사명 합치기
station_result.drop(784, inplace=True)
station_result.iloc[403, [9, 10, 11]] = (
    2738.0, 37.485196, 126.981605)

# 결과 확인
# 모든 역사명 맞추기
station_result["역사명"] = station_result["역사명"].replace({
    "이수": "총신대입구(이수)",
    "총신대입구": "총신대입구(이수)"})
station_result[station_result['역사명'] == '총신대입구(이수)']

,역번호_x,역사명,노선번호,노선명,환승역구분,환승노선번호,환승노선명,위도_x,경도_x,역번호_y,위도_y,경도_y
276,0432,총신대입구(이수),I1104,4호선,도시철도 환승역,S1107,수도권 도시철도 7호선,NaN,NaN,432.0,37.486263,126.981989
403,0736,총신대입구(이수),S1107,7호선,도시철도 환승역,I1104,수도권 광역철도 4호선,NaN,NaN,2738.0,37.485196,126.981605


In [652]:
# 9호선
station_result[(station_result['노선명'] == '9호선') & (station_result['역번호_x'].isnull())]

# 맞출 데이터 없음

,역번호_x,역사명,노선번호,노선명,환승역구분,환승노선번호,환승노선명,위도_x,경도_x,역번호_y,위도_y,경도_y
771,NaN,중앙보훈병원,NaN,9호선,NaN,NaN,NaN,NaN,NaN,4138.0,37.529191,127.148739
772,NaN,둔촌오륜,NaN,9호선,NaN,NaN,NaN,NaN,NaN,4137.0,37.519683,127.137989
773,NaN,올림픽공원(한국체대),NaN,9호선,NaN,NaN,NaN,NaN,NaN,4136.0,37.516269,127.130288
774,NaN,한성백제,NaN,9호선,NaN,NaN,NaN,NaN,NaN,4135.0,37.516404,127.116503
775,NaN,송파나루,NaN,9호선,NaN,NaN,NaN,NaN,NaN,4134.0,37.510372,127.112216
776,NaN,석촌,NaN,9호선,NaN,NaN,NaN,NaN,NaN,4133.0,37.505208,127.107040
777,NaN,석촌고분,NaN,9호선,NaN,NaN,NaN,NaN,NaN,4132.0,37.502558,127.097033
778,NaN,삼전,NaN,9호선,NaN,NaN,NaN,NaN,NaN,4131.0,37.504738,127.088025
779,NaN,종합운동장,NaN,9호선,NaN,NaN,NaN,NaN,NaN,4130.0,37.511426,127.076275
780,NaN,봉은사,NaN,9호선,NaN,NaN,NaN,NaN,NaN,4129.0,37.514219,127.060245


In [653]:
# 경부선
station_result[(station_result['노선명'] == '경부선') & (
    station_result['역번호_x'].isnull() | station_result['역번호_y'].isnull())]

# 서울역 데이터 수정 필요
station_result.drop([558, 816], inplace=True)
station_result[station_result['역사명'] == '서울역']

,역번호_x,역사명,노선번호,노선명,환승역구분,환승노선번호,환승노선명,위도_x,경도_x,역번호_y,위도_y,경도_y
92,A01,서울역,I28A1,공항철도선,환승역,S1101+S1104+I4108,서울 도시철도 1호선+서울 도시철도 4호선+경의선,37.549118,126.970464,4201.0,37.553247,126.969769
168,0133,서울역,I4101,1호선,도시철도 환승역,I1104+I41K4+I28A1,수도권 광역철도 4호선+수도권 광역철도 경의중앙+수도권 광역철도 공항,NaN,NaN,150.0,37.556228,126.972135
270,0426,서울역,I1104,4호선,도시철도 환승역,I4101+I41K4+I28A1,수도권 광역철도 1호선+수도권 광역철도 경의중앙+수도권 광역철도 공항,NaN,NaN,426.0,37.552810,126.972556
559,1251,서울역,I4108,경의중앙선,환승역,"S2601, L2604","1호선, 4호선",37.554713,126.970787,1251.0,37.557231,126.971030


In [654]:
# 경인선
station_result[(station_result['노선명'] == '경인선') & (
    station_result['역번호_x'].isnull() | station_result['역번호_y'].isnull())]

# 역사명 합치기
station_result.drop(786, inplace=True)
station_result.iloc[620, [9, 10, 11]] = (
    1821.0, 37.492433, 126.824086)

# 결과 확인
# 모든 역사명 맞추기
station_result["역사명"] = station_result["역사명"].replace({
    "온수": "온수(성공회대입구)"})
station_result[station_result['역사명'] == "온수(성공회대입구)"]

,역번호_x,역사명,노선번호,노선명,환승역구분,환승노선번호,환승노선명,위도_x,경도_x,역번호_y,위도_y,경도_y
417,0750,온수(성공회대입구),S1107,7호선,도시철도 환승역,I4101,수도권 광역철도 1호선,NaN,NaN,2752.0,37.492092,126.823023
620,1821,온수(성공회대입구),I1101,경인선,환승역,S1107,7호선,37.492267,126.823399,NaN,NaN,NaN


In [655]:
# 경춘선
station_result[(station_result['노선명'] == '경춘선') & (
    station_result['역번호_x'].isnull() | station_result['역번호_y'].isnull())]

,역번호_x,역사명,노선번호,노선명,환승역구분,환승노선번호,환승노선명,위도_x,경도_x,역번호_y,위도_y,경도_y
637,1015,회기,I41K2,경춘선,환승역,"I4108, I4101","경의중앙선, 경원선",37.589802,127.057936,NaN,NaN,NaN
638,1014,청량리,I41K2,경춘선,환승역,"S2601, I4108, I28K1","1호선, 경의중앙선, 분당선",37.580543,127.046516,NaN,NaN,NaN
639,1201,중랑,I41K2,경춘선,환승역,I4108,경의중앙선,37.594948,127.076152,NaN,NaN,NaN
640,1202,상봉,I41K2,경춘선,환승역,"S1107, I4108","7호선, 경의중앙선",37.595254,127.085853,NaN,NaN,NaN
641,1203,망우,I41K2,경춘선,환승역,I4108,경의중앙선,37.599288,127.092318,NaN,NaN,NaN


In [656]:
# 분당선
station_result[(station_result['노선명'] == '분당선') & (
    station_result['역번호_x'].isnull() | station_result['역번호_y'].isnull())]

,역번호_x,역사명,노선번호,노선명,환승역구분,환승노선번호,환승노선명,위도_x,경도_x,역번호_y,위도_y,경도_y
661,1014,청량리,I28K1,분당선,환승역,"S2601, I41K2, I4108","1호선, 경춘선, 경의중앙선",37.580543,127.046516,NaN,NaN,NaN
662,1013,왕십리,I28K1,분당선,환승역,"S2602, S1105, I4108","2호선, 5호선, 경의중앙선",37.561128,127.035505,NaN,NaN,NaN


In [657]:
# 서해선
station_result[(station_result['노선명'] == '서해선') & (
    station_result['역번호_x'].isnull() | station_result['역번호_y'].isnull())]

# 원곡역 존재하지 않음, 삭제
station_result.drop(737, inplace=True)

station_result[(station_result['노선명'] == '서해선') & (
    station_result['역번호_x'].isnull() | station_result['역번호_y'].isnull())]

,역번호_x,역사명,노선번호,노선명,환승역구분,환승노선번호,환승노선명,위도_x,경도_x,역번호_y,위도_y,경도_y
766,4814,시우,I41WS,서해선,일반역,NaN,NaN,37.313767,126.798303,NaN,NaN,NaN
768,NaN,원곡,NaN,서해선,NaN,NaN,NaN,NaN,NaN,4814.0,37.31321,126.796261


In [658]:
# 안산과천선
station_result[(station_result['노선명'] == '안산과천선') & (
    station_result['역번호_x'].isnull() | station_result['역번호_y'].isnull())]

,역번호_x,역사명,노선번호,노선명,환승역구분,환승노선번호,환승노선명,위도_x,경도_x,역번호_y,위도_y,경도_y
787,NaN,금정,NaN,안산과천선,NaN,NaN,NaN,NaN,NaN,1458.0,37.372209,126.943417


In [659]:
# 수인선
station_result[(station_result['노선명'] == '수인선') & (
    station_result['역번호_x'].isnull() | station_result['역번호_y'].isnull())]

,역번호_x,역사명,노선번호,노선명,환승역구분,환승노선번호,환승노선명,위도_x,경도_x,역번호_y,위도_y,경도_y
703,1755,한대앞,I28K1,수인선,일반역,NaN,NaN,37.309685,126.853656,NaN,NaN,NaN
704,1756,중앙,I28K1,수인선,일반역,NaN,NaN,37.316055,126.838532,NaN,NaN,NaN
705,1757,고잔,I28K1,수인선,일반역,NaN,NaN,37.316821,126.823094,NaN,NaN,NaN
706,1758,초지,I28K1,수인선,환승역,"I4103, I41WS","안산과천선, 서해선",37.320583,126.806151,NaN,NaN,NaN
707,1759,안산,I28K1,수인선,일반역,NaN,NaN,37.327006,126.788748,NaN,NaN,NaN
708,1760,신길온천,I28K1,수인선,일반역,NaN,NaN,37.337488,126.767306,NaN,NaN,NaN
709,1761,정왕,I28K1,수인선,일반역,NaN,NaN,37.351816,126.742844,NaN,NaN,NaN
710,1762,오이도,I28K1,수인선,일반역,NaN,NaN,37.361773,126.738437,NaN,NaN,NaN


In [660]:
# 에버라인선
station_result[(station_result['노선명'] == '에버라인선') & (
    station_result['역번호_x'].isnull() | station_result['역번호_y'].isnull())]

# 역사명 합치기
station_result.drop(770, inplace=True)
station_result.iloc[77, [9, 10, 11]] = (
    4501.0, 37.275449, 127.116665)

# 결과 확인
# 모든 역사명 맞추기
station_result["역사명"] = station_result["역사명"].replace({
    "기흥": "기흥(백남준아트센터)"})
station_result[station_result['역사명'] == "기흥(백남준아트센터)"]


,역번호_x,역사명,노선번호,노선명,환승역구분,환승노선번호,환승노선명,위도_x,경도_x,역번호_y,위도_y,경도_y
77,Y110,기흥(백남준아트센터),L41E1,에버라인선,환승역,L41E1,수도권 경량도시철도 에버라인,37.275453,127.117046,4501.0,37.275449,127.116665
689,1865,기흥(백남준아트센터),I28K1,분당선,환승역,L41E1,용인경전철,37.275657,127.115944,1865.0,37.275061,127.115910


In [661]:
# 우이신설선
station_result[(station_result['노선명'] == '우이신설선') & (
    station_result['역번호_x'].isnull() | station_result['역번호_y'].isnull())]

# 역사명 합치기
station_result.drop(769, inplace=True)
station_result.iloc[116, [9, 10, 11]] = (
    4711.0, 37.592467, 127.016516)

# 결과 확인
# 모든 역사명 맞추기
station_result["역사명"] = station_result["역사명"].replace({
    "성신여대입구": "성신여대입구(돈암)",
    "돈암": "성신여대입구(돈암)"})
station_result[station_result['역사명'] == "성신여대입구(돈암)"]

,역번호_x,역사명,노선번호,노선명,환승역구분,환승노선번호,환승노선명,위도_x,경도_x,역번호_y,위도_y,경도_y
116,S120,성신여대입구(돈암),L11UI,우이신설선,도시철도 환승역,I1104,수도권 광역철도 4호선,NaN,NaN,4711.0,37.592467,127.016516
262,0418,성신여대입구(돈암),I1104,4호선,도시철도 일반역,I1104,수도권 광역철도 4호선,NaN,NaN,418.0,37.592612,127.016441


In [662]:
# 경원선
print(len(station_result[(station_result['노선명'] == '경원선') & (
    station_result['역번호_x'].isnull())]))
station_one = station_result[(station_result['노선명'] == '경원선') & (
    station_result['역번호_x'].isnull())]

station_one = station_one.replace({'경원선': '경의중앙선'})

station_one

7


,역번호_x,역사명,노선번호,노선명,환승역구분,환승노선번호,환승노선명,위도_x,경도_x,역번호_y,위도_y,경도_y
809,NaN,청량리(서울시립대입구),NaN,경의중앙선,NaN,NaN,NaN,NaN,NaN,1014.0,37.580759,127.048300
810,NaN,왕십리(성동구청),NaN,경의중앙선,NaN,NaN,NaN,NaN,NaN,1013.0,37.561827,127.038352
811,NaN,응봉,NaN,경의중앙선,NaN,NaN,NaN,NaN,NaN,1012.0,37.549946,127.034538
812,NaN,옥수,NaN,경의중앙선,NaN,NaN,NaN,NaN,NaN,1011.0,37.540446,127.018672
813,NaN,한남,NaN,경의중앙선,NaN,NaN,NaN,NaN,NaN,1010.0,37.529430,127.009169
814,NaN,서빙고,NaN,경의중앙선,NaN,NaN,NaN,NaN,NaN,1009.0,37.519594,126.988537
815,NaN,이촌(국립중앙박물관),NaN,경의중앙선,NaN,NaN,NaN,NaN,NaN,1008.0,37.522427,126.973406


In [663]:
# 중앙선
print(len(station_result[(station_result['노선명'] == '중앙선') & (
    station_result['역번호_x'].isnull())]))
station_cau = station_result[(station_result['노선명'] == '중앙선') & (
    station_result['역번호_x'].isnull() | station_result['역번호_y'].isnull())]

station_cau = station_cau.replace({'중앙선': '경의중앙선'})

station_cau

21


,역번호_x,역사명,노선번호,노선명,환승역구분,환승노선번호,환승노선명,위도_x,경도_x,역번호_y,위도_y,경도_y
788,NaN,지평,NaN,경의중앙선,NaN,NaN,NaN,NaN,NaN,1220.0,37.476393,127.629874
789,NaN,용문,NaN,경의중앙선,NaN,NaN,NaN,NaN,NaN,1219.0,37.482230,127.594647
790,NaN,원덕,NaN,경의중앙선,NaN,NaN,NaN,NaN,NaN,1218.0,37.468672,127.547076
791,NaN,양평,NaN,경의중앙선,NaN,NaN,NaN,NaN,NaN,1217.0,37.492773,127.491837
792,NaN,오빈,NaN,경의중앙선,NaN,NaN,NaN,NaN,NaN,1216.0,37.506062,127.473868
793,NaN,아신,NaN,경의중앙선,NaN,NaN,NaN,NaN,NaN,1215.0,37.513820,127.443173
794,NaN,국수,NaN,경의중앙선,NaN,NaN,NaN,NaN,NaN,1214.0,37.516169,127.399367
795,NaN,신원,NaN,경의중앙선,NaN,NaN,NaN,NaN,NaN,1213.0,37.525545,127.372921
796,NaN,양수,NaN,경의중앙선,NaN,NaN,NaN,NaN,NaN,1212.0,37.545981,127.329098
797,NaN,운길산,NaN,경의중앙선,NaN,NaN,NaN,NaN,NaN,1211.0,37.554669,127.310115


In [664]:
# 경의중앙선
print(len(station_result[(station_result['노선명'] == '경의중앙선') & (
    station_result['역번호_y'].isnull())]))
station_gyeong = station_result[(station_result['노선명'] == '경의중앙선') & (
    station_result['역번호_x'].isnull() | station_result['역번호_y'].isnull())]

# y 컬럼 제거
station_gyeong = station_gyeong.drop(columns=['역번호_y', '위도_y', '경도_y'])

# 역이름 변경
station_gyeong = station_gyeong.replace({
    '청량리': '청량리(서울시립대입구)',
    '왕십리': '왕십리(성동구청)',
    '이촌': '이촌(국립중앙박물관)',
    '상봉': '상봉(시외버스터미널)'
})

station_gyeong

30


,역번호_x,역사명,노선번호,노선명,환승역구분,환승노선번호,환승노선명,위도_x,경도_x
562,1003,용산,I4108,경의중앙선,환승역,I4101,경부선,37.529890,126.964775
582,1013,왕십리(성동구청),I4108,경의중앙선,환승역,"S2602, S1105, I28K1","2호선, 5호선, 분당선",37.561128,127.035505
583,1014,청량리(서울시립대입구),I4108,경의중앙선,환승역,"S2601, I41K2, I28K1","1호선, 경춘선, 분당선",37.580543,127.046516
584,1015,회기,I4108,경의중앙선,환승역,"I4101, I41K2","경원선, 경춘선",37.589802,127.057936
585,1201,중랑,I4108,경의중앙선,환승역,I41K2,경춘선,37.594948,127.076152
586,1202,상봉(시외버스터미널),I4108,경의중앙선,환승역,"S1107, I41K2","7호선, 경춘선",37.595254,127.085853
587,1203,망우,I4108,경의중앙선,환승역,I41K2,경춘선,37.599288,127.092318
588,1204,양원,I4108,경의중앙선,일반역,NaN,NaN,36.963209,129.091131
589,1220,지평,I4108,경의중앙선,일반역,NaN,NaN,37.476496,127.629681
590,1205,구리,I4108,경의중앙선,일반역,NaN,NaN,37.503178,126.882037


In [665]:
# 경원선 + 중앙선 = 경의중앙선 으로 합치기
station_temp = pd.concat([station_one, station_cau], axis = 0)

# x 컬럼 제거
station_temp = station_temp.drop(columns=['역번호_x', '노선번호', '환승역구분', '환승노선번호', '환승노선명', '위도_x', '경도_x'])

# merge
station_temp = pd.merge(station_gyeong, station_temp, on=['역사명', '노선명'], how='outer')

station_temp

,역번호_x,역사명,노선번호,노선명,환승역구분,환승노선번호,환승노선명,위도_x,경도_x,역번호_y,위도_y,경도_y
0,1003,용산,I4108,경의중앙선,환승역,I4101,경부선,37.529890,126.964775,NaN,NaN,NaN
1,1013,왕십리(성동구청),I4108,경의중앙선,환승역,"S2602, S1105, I28K1","2호선, 5호선, 분당선",37.561128,127.035505,1013.0,37.561827,127.038352
2,1014,청량리(서울시립대입구),I4108,경의중앙선,환승역,"S2601, I41K2, I28K1","1호선, 경춘선, 분당선",37.580543,127.046516,1014.0,37.580759,127.048300
3,1015,회기,I4108,경의중앙선,환승역,"I4101, I41K2","경원선, 경춘선",37.589802,127.057936,1015.0,37.589460,127.057583
4,1201,중랑,I4108,경의중앙선,환승역,I41K2,경춘선,37.594948,127.076152,1201.0,37.594917,127.076116
5,1202,상봉(시외버스터미널),I4108,경의중앙선,환승역,"S1107, I41K2","7호선, 경춘선",37.595254,127.085853,1202.0,37.596678,127.085040
6,1203,망우,I4108,경의중앙선,환승역,I41K2,경춘선,37.599288,127.092318,1203.0,37.599550,127.091909
7,1204,양원,I4108,경의중앙선,일반역,NaN,NaN,36.963209,129.091131,1204.0,37.606596,127.107906
8,1220,지평,I4108,경의중앙선,일반역,NaN,NaN,37.476496,127.629681,1220.0,37.476393,127.629874
9,1205,구리,I4108,경의중앙선,일반역,NaN,NaN,37.503178,126.882037,1205.0,37.603392,127.143869


In [666]:
# 기존 데이터에 추가, 기존에 있던 애들은 삭제
station_result.drop(index=station_one.index, inplace=True)
station_result.drop(index=station_cau.index, inplace=True)
station_result.drop(index=station_gyeong.index, inplace=True)
station_result = pd.concat([station_result, station_temp])

station_result = station_result.reset_index(drop=True)
station_result

,역번호_x,역사명,노선번호,노선명,환승역구분,환승노선번호,환승노선명,위도_x,경도_x,역번호_y,위도_y,경도_y
0,3110,계양,S2801,인천1호선,환승역,S2801+I28A1,공항철도,37.571539,126.736319,3110.0,37.571449,126.735780
1,3111,귤현,S2801,인천1호선,일반역,-,-,37.566362,126.742498,3111.0,37.566379,126.742654
2,3112,박촌,S2801,인천1호선,일반역,-,-,37.553525,126.744946,3112.0,37.553703,126.745077
3,3113,임학,S2801,인천1호선,일반역,-,-,37.545058,126.738642,3113.0,37.545059,126.738665
4,3114,계산,S2801,인천1호선,일반역,-,-,37.543243,126.728436,3114.0,37.543238,126.728128
...,...,...,...,...,...,...,...,...,...,...,...,...
776,1008,이촌(국립중앙박물관),I4108,경의중앙선,환승역,L2604,4호선,37.522476,126.973816,1008.0,37.522427,126.973406
777,1009,서빙고,I4108,경의중앙선,일반역,NaN,NaN,37.519610,126.988358,1009.0,37.519594,126.988537
778,1010,한남,I4108,경의중앙선,일반역,NaN,NaN,37.529361,127.008984,1010.0,37.529430,127.009169
779,1011,옥수,I4108,경의중앙선,환승역,S2603,3호선,37.540514,127.018698,1011.0,37.540446,127.018672


##### 3-b. 위경도값 하나로 합치기

In [667]:
# x, y에 대해, 둘 중 한 곳에 없는 위경도값이 각각 몇 개인지 count
lat_in_only_x = station_result[station_result["위도_y"].isnull(
) & station_result["위도_x"].notnull()]
lng_in_only_x = station_result[station_result["경도_y"].isnull(
) & station_result["경도_x"].notnull()]

lat_in_only_x_cnt = len(lat_in_only_x)
lng_in_only_x_cnt = len(lng_in_only_x)

print("x에만 있는 위경도값 개수 :", lat_in_only_x_cnt, lng_in_only_x_cnt)

lat_in_only_y = station_result[station_result["위도_x"].isnull(
) & station_result["위도_y"].notnull()]
lng_in_only_y = station_result[station_result["경도_x"].isnull(
) & station_result["경도_y"].notnull()]

lat_in_only_y_cnt = len(lat_in_only_y)
lng_in_only_y_cnt = len(lng_in_only_y)

print("y에만 있는 위경도값 개수 :", lat_in_only_y_cnt, lng_in_only_y_cnt)

# 한쪽 위경도가 NaN이면, 반대쪽 위경도 넣어주기
station_result["위도_x"] = station_result["위도_x"].fillna(station_result["위도_y"])
station_result["위도_y"] = station_result["위도_y"].fillna(station_result["위도_x"])
station_result["경도_x"] = station_result["경도_x"].fillna(station_result["경도_y"])
station_result["경도_y"] = station_result["경도_y"].fillna(station_result["경도_x"])

# 위경도 데이터의 오차 계산
station_result["위도차"] = (station_result["위도_x"] - station_result["위도_y"]).abs()
station_result["경도차"] = (station_result["경도_x"] - station_result["경도_y"]).abs()

x에만 있는 위경도값 개수 : 27 27
y에만 있는 위경도값 개수 : 378 378


In [668]:
# 위도 오차값 확인
lat_diff_mean = station_result["위도차"].mean()
print(lat_diff_mean)

# 평균보다 오차 큰 행 확인
bigger_lat_diff = station_result[station_result["위도차"] >= lat_diff_mean]
bigger_lat_diff

1.841841648931754


,역번호_x,역사명,노선번호,노선명,환승역구분,환승노선번호,환승노선명,위도_x,경도_x,역번호_y,위도_y,경도_y,위도차,경도차
119,D004,신사,I11D1,신분당선,도시철도 환승역,I11D1,수도권 광역철도 신분당선,127.019761,37.516125,4304.0,37.516334,127.020114,89.503427,89.503989
120,D005,논현,I11D1,신분당선,도시철도 환승역,I11D1,수도권 광역철도 신분당선,127.021366,37.511115,4305.0,37.511093,127.021415,89.510273,89.510300
121,D006,신논현,I11D1,신분당선,도시철도 환승역,I11D1,수도권 광역철도 신분당선,127.024531,37.504555,4306.0,37.504598,127.025060,89.519933,89.520505
122,D007,강남,I11D1,신분당선,도시철도 환승역,I11D1,수도권 광역철도 신분당선,127.027879,37.497385,4307.0,37.496837,127.028104,89.531042,89.530719
123,D008,양재(서초구청),I11D1,신분당선,도시철도 환승역,I11D1,수도권 광역철도 신분당선,127.034099,37.484454,4308.0,37.483809,127.034653,89.550290,89.550199
124,D009,양재시민의숲(매헌),I11D1,신분당선,도시철도 일반역,NaN,NaN,127.038420,37.470964,4309.0,37.470023,127.038420,89.568397,89.567456
125,D010,청계산입구,I11D1,신분당선,도시철도 일반역,NaN,NaN,127.054340,37.448715,4310.0,37.447211,127.055664,89.607129,89.606949
126,D011,판교,I11D1,신분당선,도시철도 일반역,I11D1,수도권 광역철도 신분당선,127.111211,37.395715,4311.0,37.394761,127.112217,89.716450,89.716502
127,D012,정자,I11D1,신분당선,도시철도 환승역,I11D1,수도권 광역철도 신분당선,127.108386,37.368210,4312.0,37.367098,127.108403,89.741288,89.740193
128,D013,미금,I11D1,신분당선,도시철도 일반역,I11D1,수도권 광역철도 신분당선,127.108903,37.351183,4313.0,37.349982,127.108918,89.758921,89.757735


In [669]:
# 경도 오차값 확인
lng_diff_mean = station_result["경도차"].mean()
print(lng_diff_mean)

# 평균보다 오차 큰 행 확인
bigger_lng_diff = station_result[station_result["경도차"] >= lng_diff_mean]
bigger_lng_diff

1.843793911888348


,역번호_x,역사명,노선번호,노선명,환승역구분,환승노선번호,환승노선명,위도_x,경도_x,역번호_y,위도_y,경도_y,위도차,경도차
54,3225,남동구청,S2802,인천2호선,일반역,-,-,37.448179,123.736581,3225.0,37.448161,126.736939,0.000018,3.000358
119,D004,신사,I11D1,신분당선,도시철도 환승역,I11D1,수도권 광역철도 신분당선,127.019761,37.516125,4304.0,37.516334,127.020114,89.503427,89.503989
120,D005,논현,I11D1,신분당선,도시철도 환승역,I11D1,수도권 광역철도 신분당선,127.021366,37.511115,4305.0,37.511093,127.021415,89.510273,89.510300
121,D006,신논현,I11D1,신분당선,도시철도 환승역,I11D1,수도권 광역철도 신분당선,127.024531,37.504555,4306.0,37.504598,127.025060,89.519933,89.520505
122,D007,강남,I11D1,신분당선,도시철도 환승역,I11D1,수도권 광역철도 신분당선,127.027879,37.497385,4307.0,37.496837,127.028104,89.531042,89.530719
123,D008,양재(서초구청),I11D1,신분당선,도시철도 환승역,I11D1,수도권 광역철도 신분당선,127.034099,37.484454,4308.0,37.483809,127.034653,89.550290,89.550199
124,D009,양재시민의숲(매헌),I11D1,신분당선,도시철도 일반역,NaN,NaN,127.038420,37.470964,4309.0,37.470023,127.038420,89.568397,89.567456
125,D010,청계산입구,I11D1,신분당선,도시철도 일반역,NaN,NaN,127.054340,37.448715,4310.0,37.447211,127.055664,89.607129,89.606949
126,D011,판교,I11D1,신분당선,도시철도 일반역,I11D1,수도권 광역철도 신분당선,127.111211,37.395715,4311.0,37.394761,127.112217,89.716450,89.716502
127,D012,정자,I11D1,신분당선,도시철도 환승역,I11D1,수도권 광역철도 신분당선,127.108386,37.368210,4312.0,37.367098,127.108403,89.741288,89.740193


In [670]:
# 119 ~ 134 위도_x와 경도_x가 뒤바뀌어 들어감
# 송도, (위도_x, 경도_x) 전남 해남에 위치
# 남동구청, (위도_x, 경도_x) 황해에 위치
# 양원, (위도_x, 경도_x) 양원역 위치가 경북 봉화군에 위치한 양원역 위치로 잘못 들어가져 있음
# -> 위경도 데이터의 경우, x를 y로 덮어쓰면 될 것 같음
# 이를 증명하기 위해, 위의 데이터들의 위도차/경도차가 해결된 뒤,
# 위경도차가 높은 행들을 각각 확인하여, 거리상의 오차가 어느 정도일지 확인해볼 것

station_result.loc[[119, 120, 121, 122, 123, 124, 125, 126,
                    127, 128, 129, 130, 131, 132, 133, 134, 688, 54, 758], ['위도차', '경도차']] = 0.0

In [671]:
# 위도 오차값 평균 재확인
lat_diff_mean = station_result["위도차"].mean()
print(lat_diff_mean)

# 평균보다 오차 큰 행 확인
bigger_lat_diff = station_result[station_result["위도차"] >= lat_diff_mean]
bigger_lat_diff.sort_values('위도차').tail()

0.00031386301626111214


,역번호_x,역사명,노선번호,노선명,환승역구분,환승노선번호,환승노선명,위도_x,경도_x,역번호_y,위도_y,경도_y,위도차,경도차
44,3215,인천가좌,S2802,인천2호선,일반역,-,-,37.493762,126.675465,3215.0,37.489700,126.675208,0.004062,0.000257
92,A01,서울역,I28A1,공항철도선,환승역,S1101+S1104+I4108,서울 도시철도 1호선+서울 도시철도 4호선+경의선,37.549118,126.970464,4201.0,37.553247,126.969769,0.004129,0.000695
590,1803,역곡,I1101,경인선,일반역,NaN,NaN,37.485140,126.811622,1821.0,37.492433,126.824086,0.007293,0.012464
570,1268,화전,I4108,경의중앙선,일반역,NaN,NaN,37.637837,126.832503,1268.0,37.602888,126.868387,0.034949,0.035884
760,1205,구리,I4108,경의중앙선,일반역,NaN,NaN,37.503178,126.882037,1205.0,37.603392,127.143869,0.100214,0.261832


In [672]:
# 경도 오차값 평균 재확인
lng_diff_mean = station_result["경도차"].mean()
print(lng_diff_mean)

# 평균보다 오차 큰 행 확인
bigger_lng_diff = station_result[station_result["경도차"] >= lng_diff_mean]
bigger_lng_diff.sort_values('경도차').tail()

0.0005531796732392762


,역번호_x,역사명,노선번호,노선명,환승역구분,환승노선번호,환승노선명,위도_x,경도_x,역번호_y,위도_y,경도_y,위도차,경도차
670,1876,야목,I28K1,수인선,일반역,NaN,NaN,37.261011,126.884290,1876.0,37.264179,126.879483,0.003168,0.004807
590,1803,역곡,I1101,경인선,일반역,NaN,NaN,37.485140,126.811622,1821.0,37.492433,126.824086,0.007293,0.012464
105,A11,인천공항2터미널,I28A1,공항철도선,일반역,NaN,NaN,37.458269,126.427536,4215.0,37.460699,126.441442,0.002430,0.013906
570,1268,화전,I4108,경의중앙선,일반역,NaN,NaN,37.637837,126.832503,1268.0,37.602888,126.868387,0.034949,0.035884
760,1205,구리,I4108,경의중앙선,일반역,NaN,NaN,37.503178,126.882037,1205.0,37.603392,127.143869,0.100214,0.261832


##### 3-b. 위경도 결측값 재확인

In [673]:
# 구리, (위도_x, 경도_x) 구로 들어가 있음
# 화전, (위도_x, 경도_x) 덕양구청에 위치, 도보 1시간 30분의 오차
# 그 다음으로 오차가 큰 인천공항2터미널과 서울역의 경우 역이 워낙 커서 생기는 일, 문제 없음
# 즉, x와 y 두 곳에 있는 값들에 대해서는 이상값 검토가 끝남 
# 둘 다에 있는 값의 경우 그냥 y쪽 값으로 덮어쓰기
# 둘 중 한 곳에만 있는 위경도값들은 위에서 따로 저장해 둠, 
# 이후 지도를 보며 재검토할 예정  

# y에 없는 값들 x에서 가져와 채우기
station_result['위도_y'] = station_result['위도_y'].fillna(station_result['위도_x'])
station_result['경도_y'] = station_result['경도_y'].fillna(station_result['경도_x'])

# 개수 재확인
null_lat_in_y = station_result[station_result["위도_y"].isnull()]
null_lng_in_y = station_result[station_result["경도_y"].isnull()]
null_lat_in_y_cnt = len(null_lat_in_y)
null_lng_in_y_cnt = len(null_lng_in_y)

print("y에 있는 위경도 결측값 개수 :", null_lat_in_y_cnt, null_lng_in_y_cnt)

null_lat_in_y 

y에 있는 위경도 결측값 개수 : 0 0


,역번호_x,역사명,노선번호,노선명,환승역구분,환승노선번호,환승노선명,위도_x,경도_x,역번호_y,위도_y,경도_y,위도차,경도차


In [674]:
# 위도_x, 경도_x 삭제 및 위도_y와 경도_y의 이름 변경
station_result.drop(columns=["위도_x", "경도_x", "위도차", "경도차"], inplace=True)
station_result.rename(columns={'위도_y': '위도', '경도_y': '경도'}, inplace=True)

station_result

,역번호_x,역사명,노선번호,노선명,환승역구분,환승노선번호,환승노선명,역번호_y,위도,경도
0,3110,계양,S2801,인천1호선,환승역,S2801+I28A1,공항철도,3110.0,37.571449,126.735780
1,3111,귤현,S2801,인천1호선,일반역,-,-,3111.0,37.566379,126.742654
2,3112,박촌,S2801,인천1호선,일반역,-,-,3112.0,37.553703,126.745077
3,3113,임학,S2801,인천1호선,일반역,-,-,3113.0,37.545059,126.738665
4,3114,계산,S2801,인천1호선,일반역,-,-,3114.0,37.543238,126.728128
...,...,...,...,...,...,...,...,...,...,...
776,1008,이촌(국립중앙박물관),I4108,경의중앙선,환승역,L2604,4호선,1008.0,37.522427,126.973406
777,1009,서빙고,I4108,경의중앙선,일반역,NaN,NaN,1009.0,37.519594,126.988537
778,1010,한남,I4108,경의중앙선,일반역,NaN,NaN,1010.0,37.529430,127.009169
779,1011,옥수,I4108,경의중앙선,환승역,S2603,3호선,1011.0,37.540446,127.018672


### 4. 지도에 시각화

##### 4-a. 이상값 확인

In [675]:
# 지도 시각화 
geo_station = station_result.copy()

# 중심 설정
fmap = folium.Map(location=[geo_station['위도'].mean(
), geo_station['경도'].mean()], zoom_start=0, width=750, height=500)

for n in geo_station.index:
    # 팝업에 들어갈 텍스트를 지정
    popup_name = str(geo_station.loc[n, '위도']) + ", " + str(geo_station.loc[n, '경도']) + \
        " \n" + geo_station.loc[n, '역사명'] + ' - ' + geo_station.loc[n, '노선명']
    
    folium.Marker(
        location=[geo_station.loc[n, '위도'], geo_station.loc[n, '경도']],
        popup=popup_name, 
        tooltip=popup_name
    ).add_to(fmap)

fmap

In [678]:
# 위경도가 한국을 벗어난 역들 찾아보기
station_outof_korea = station_result[(station_result['위도'] < 33.10000000) |
                                     (station_result['위도'] > 38.45000000) | (station_result['경도'] < 124.19583333) |
                                     (station_result['경도'] > 131.87222222)]

station_outof_korea

# 확인 결과, 모든 역이 제대로 한국 내에 존재함

,역번호_x,역사명,노선번호,노선명,환승역구분,환승노선번호,환승노선명,역번호_y,위도,경도


In [679]:
# 더 크게 지도 시각화 
geo_station = station_result.copy()

# 중심 설정
fmap = folium.Map(location=[geo_station['위도'].mean(
), geo_station['경도'].mean()], zoom_start=8, width=750, height=500)

for n in geo_station.index:
    # 팝업에 들어갈 텍스트를 지정
    popup_name = str(geo_station.loc[n, '위도']) + ", " + str(geo_station.loc[n, '경도']
                                                            ) + " \n" + geo_station.loc[n, '역사명'] + ' - ' + geo_station.loc[n, '노선명']

    folium.Marker(
        location=[geo_station.loc[n, '위도'], geo_station.loc[n, '경도']],
        popup=popup_name,
        tooltip=popup_name
    ).add_to(fmap)

fmap

### 5. 역번호, 노선번호, 역이름, 노선명 맞추기

##### 5-a. 서울시 지하철 실시간 도착정보에 쓰이는 역 및 노선 종류 확인

In [680]:
# 데이터 읽어오기
station_num_name_seoul = pd.read_csv('train_station_num_name_seoul.csv')

# 컬럼명 변경
station_num_name_seoul.columns=["역번호", "역사명", "노선명", "외부코드"]

station_num_name_seoul

,역번호,역사명,노선명,외부코드
0,244,용답,02호선,211-1
1,245,신답,02호선,211-2
2,250,용두,02호선,211-3
3,336,학여울,03호선,346
4,428,삼각지,04호선,428
...,...,...,...,...
762,159,동묘앞,01호선,127
763,200,까치산,02호선,234-4
764,201,시청,02호선,201
765,202,을지로입구,02호선,202


In [681]:
# 결측값 존재 확인
station_num_name_seoul.info()

# 확인 결과, 외부코드를 제외한 모든 값이 잘 들어있음 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 767 entries, 0 to 766
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   역번호     767 non-null    object
 1   역사명     767 non-null    object
 2   노선명     767 non-null    object
 3   외부코드    756 non-null    object
dtypes: object(4)
memory usage: 24.1+ KB


In [682]:
# 해당 데이터에 포함된 노선명 확인 및 둘 중 한 데이터셋에만 있는 노선명 확인
route_name_result = set(station_result['노선명'])
route_name_seoul = set(station_num_name_seoul["노선명"])
sub_route_name_result = route_name_result - route_name_seoul
sub_route_name_seoul = route_name_seoul - route_name_result

print(route_name_result)
print(route_name_seoul)

print(sub_route_name_result)
print(sub_route_name_seoul)

# 확인 결과, 공항철도, 경춘선 이름을 맞출 필요 있음
# 수인분당선, 경의선의 경우 다른, 다른 모든 API 확인한 뒤 통합할지 쪼갤지 고민 필요

{'경부선', '3호선', '일산선', '2호선', '신분당선', '7호선', '6호선', '의정부선', '수인선', '8호선', '경인선', '에버라인선', '김포골드라인', '경강선', '9호선', '4호선', '서해선', '신림선', '경춘선', '1호선', '안산과천선', '인천1호선', '경원선', '분당선', '진접선', '인천2호선', '5호선', '공항철도선', '경의중앙선', '우이신설선'}
{'우이신설경전철', '김포도시철도', '신분당선', '07호선', '02호선', '01호선', '인천선', '08호선', '09호선', '경강선', '서해선', '신림선', '의정부경전철', '수인분당선', '경춘선', '경의선', '용인경전철', '인천2호선', '공항철도', '05호선', '04호선', '06호선', '03호선'}
{'경부선', '3호선', '일산선', '2호선', '7호선', '6호선', '의정부선', '수인선', '경인선', '8호선', '에버라인선', '김포골드라인', '9호선', '4호선', '1호선', '안산과천선', '인천1호선', '경원선', '분당선', '진접선', '5호선', '공항철도선', '경의중앙선', '우이신설선'}
{'02호선', '의정부경전철', '수인분당선', '01호선', '우이신설경전철', '공항철도', '05호선', '04호선', '김포도시철도', '06호선', '인천선', '경의선', '08호선', '용인경전철', '09호선', '03호선', '07호선'}


In [683]:
# 수인분당선, 경의중앙선을 제외한 부분만 일단 통일
station_num_name_seoul["노선명"] = station_num_name_seoul["노선명"].replace({
    "우이신설경전철": "우이신설선",
    "의정부경전철": "의정부선",
    "용인경전철": "에버라인선",
    "김포도시철도": "김포골드라인",
    "인천선": "인천1호선"
})
station_num_name_seoul["노선명"] = station_num_name_seoul["노선명"].str.replace(
    pat=r'^0', repl=r'', regex=True)
station_result["노선명"] = station_result["노선명"].replace({
    "공항철도1호선": "공항철도"
})

# 둘 중 한 데이터셋에만 있는 노선명 재확인
route_name_result = set(station_result['노선명'])
route_name_seoul = set(station_num_name_seoul["노선명"])
sub_route_name_result = route_name_result - route_name_seoul
sub_route_name_seoul = route_name_seoul - route_name_result

print(route_name_result)
print(route_name_seoul)

print(sub_route_name_result)
print(sub_route_name_seoul)

{'경부선', '3호선', '일산선', '2호선', '신분당선', '7호선', '6호선', '의정부선', '수인선', '8호선', '경인선', '에버라인선', '김포골드라인', '경강선', '9호선', '4호선', '서해선', '신림선', '경춘선', '1호선', '안산과천선', '인천1호선', '경원선', '분당선', '진접선', '인천2호선', '5호선', '공항철도선', '경의중앙선', '우이신설선'}
{'3호선', '2호선', '신분당선', '7호선', '6호선', '의정부선', '8호선', '에버라인선', '김포골드라인', '경강선', '9호선', '4호선', '서해선', '신림선', '수인분당선', '1호선', '경춘선', '인천1호선', '경의선', '인천2호선', '공항철도', '5호선', '우이신설선'}
{'경부선', '안산과천선', '경원선', '공항철도선', '일산선', '수인선', '경인선', '경의중앙선', '분당선', '진접선'}
{'공항철도', '경의선', '수인분당선'}


##### 5-b. 전체 지하철 시간표 정보에 쓰이는 역 및 노선 종류 확인

In [684]:
# 데이터 읽어오기
station_num_name = pd.read_csv('train_station_num_name.csv')

# 컬럼명 변경
station_num_name = station_num_name.rename(
    columns={"LN_CD": "노선번호", "LN_NM": "노선명", "STIN_CD": "역번호", "STIN_NM": "역사명"})

# 필요 없는 노선 제거
station_num_name = station_num_name[(station_num_name['노선명'] != '동해선')]
station_num_name = station_num_name[(station_num_name['노선명'] != '부산김해경전철')]
station_num_name = station_num_name[(station_num_name['노선명'] != '자기부상')]

station_num_name = station_num_name.reset_index(drop=True)
station_num_name  

,RAIL_OPR_ISTT_CD,RAIL_OPR_ISTT_NM,노선번호,노선명,역번호,역사명
0,AR,공항철도주식회사,A1,공항철도,A01,서울역
1,AR,공항철도주식회사,A1,공항철도,A02,공덕
2,AR,공항철도주식회사,A1,공항철도,A03,홍대입구
3,AR,공항철도주식회사,A1,공항철도,A04,디지털미디어시티
4,AR,공항철도주식회사,A1,공항철도,A042,마곡나루
...,...,...,...,...,...,...
1011,DJ,대전교통공사,1,1호선,0018,현충원(한밭대)
1012,DJ,대전교통공사,1,1호선,0019,월드컵경기장(노은도매시장)
1013,DJ,대전교통공사,1,1호선,0020,노은
1014,DJ,대전교통공사,1,1호선,0021,지족(침신대)


In [685]:
# 결측값 존재 확인
station_num_name.info()

# 확인 결과, 모든 값이 제대로 잘 들어있음

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1016 entries, 0 to 1015
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   RAIL_OPR_ISTT_CD  1016 non-null   object
 1   RAIL_OPR_ISTT_NM  1016 non-null   object
 2   노선번호              1016 non-null   object
 3   노선명               1016 non-null   object
 4   역번호               1016 non-null   object
 5   역사명               1016 non-null   object
dtypes: object(6)
memory usage: 47.8+ KB


In [686]:
# 해당 데이터에 포함된 노선명 확인 및 둘 중 한 데이터셋에만 있는 노선명 확인
route_name_result = set(station_result['노선명'])
route_name = set(station_num_name["노선명"])
sub_route_name_result = route_name_result - route_name
sub_route_name = route_name - route_name_result

print(sub_route_name_result)
print(sub_route_name)

{'경부선', '경춘선', '안산과천선', '의정부선', '경원선', '공항철도선', '일산선', '경의중앙선', '수인선', '경인선', '에버라인선', '우이신설선', '분당선', '진접선'}
{'의정부경전철', '공항철도', '경춘', '수인분당', '경의중앙', '용인에버라인', '우이신설'}


In [687]:
# 수인분당선, 경의중앙선을 제외한 부분만 일단 통일
station_num_name["노선명"] = station_num_name["노선명"].replace({
    "의정부경전철": '의정부선',
    "용인에버라인": "에버라인선",
    "수인분당": "수인분당선",
    "경춘": "경춘선",
    "경의중앙": "경의중앙선", 
    "우이신설": "우이신설선"
})

# 노선명 재확인
route_name_result = set(station_result['노선명'])
route_name = set(station_num_name["노선명"])
sub_route_name_result = route_name_result - route_name
sub_route_name = route_name - route_name_result

print(sub_route_name_result)
print(sub_route_name)

{'경부선', '안산과천선', '경원선', '공항철도선', '일산선', '수인선', '경인선', '분당선', '진접선'}
{'공항철도', '수인분당선'}
